<p align="center">
  <img src="../logo_ufjf.png" width="200"><br>
  <b>UNIVERSIDADE FEDERAL DE JUIZ DE FORA</b><br>
  Programa de Pós-Graduação em Matemática<br>
  Análise Numérica I<br><br>
  Luís Karlos Mendes de Oliveira<br><br>
  <b>Atividade 02:</b> Métodos Numéricos<br><br>
  <p align="left">Prof.: Sandro Rodrigues Mazorche</p>
  <p align="right">29 de setembro de 2025</p>
</p>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import pandas as pd
import mylib as m
import os
import ipynbname

diretorio_tabela = f"tables/{ipynbname.name()}"
diretorio_imagem = f"img/{ipynbname.name()}"
os.makedirs(diretorio_tabela, exist_ok=True)
os.makedirs(diretorio_imagem, exist_ok=True)


## **Questão 1 - d)**
$\begin{cases}
  x'(t) = 2t + e^{-tx(t)} \\
  x(0) = 1
\end{cases}$

Solução exata: $x(t) = $


In [ ]:
f = lambda t, x: 2*t + np.exp(-t*x)
condicao_inicial = [0, 1]

def x_exato(t):
    T = max(t) if isinstance(t, (np.ndarray, list)) else t
    dt = 0.001
    t_rk, x_rk = m.metodo_rk6(f, condicao_inicial, T, dt)

    return np.interp(t, t_rk, x_rk)

T = .4
h = [.1, .05]
tempos_desejados = [.1, .2, .3, .4]


### Método de Euler Explícito

In [ ]:
resultados_mee = {}
for dt in h:
  print(f'Resolvendo o PVI com h = {dt} ...')
  t_euler, y_euler = m.metodo_euler_explicito(f, condicao_inicial, T, dt)
  y_exato = x_exato(t_euler)

  dados = pd.DataFrame({
    't': t_euler,
    'x_exato(t)': y_exato,
    'x(t)': y_euler,
    'err_loc': m.erro_local(y_exato ,y_euler),
    'err_rel (%)': m.erro_relativo(y_exato ,y_euler)*100
  })

  resultados_mee[f'h={dt}'] = dados

for nome, df in resultados_mee.items():
  mascara = df['t'].round(5).isin(tempos_desejados)
  print("\n" + "="*60)
  print(f"Resultados para {nome}")
  print("="*60)
  print(df[mascara])

  codigo_latex = df[mascara].to_latex(
    escape=True,
    index=False,
    caption=f"Resultados da simulação MEE para {nome} no intervalo [0, {T}].",
    label=f"tab:{nome.replace('=', '_')}",
    column_format="ccccc",
    float_format="%.9f"
  )

  codigo_latex = (
    "\\begin{table}[H]\n"
    "\\centering\n"
    + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
    + "\\end{table}"
  )

  nome_arquivo = f"{diretorio_tabela}/tabela_euler_{nome.replace('=', '_')}.tex"

  with open(nome_arquivo, 'w') as file:
    file.write(codigo_latex)

  print(f"✅ Tabela salva com sucesso no arquivo: {nome_arquivo}")

fig, ax = plt.subplots(figsize=(12, 7))
t_curva_exata = np.linspace(0, T, 200)
y_curva_exata = x_exato(t_curva_exata)
ax.plot(t_curva_exata, y_curva_exata, 'k-', label='Solução Aproximada (RK6)')
for nome, df in resultados_mee.items():
    ax.plot(df['t'], df['x(t)'], '.-', label=f'{nome}')

ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
nome_arquivo = f"{diretorio_imagem}/euler_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método da Série de Taylor

In [ ]:
t = sp.symbols('t')
x = sp.Function('x')(t)
f_expr = 2*t + sp.exp(-t*x)

# Calcula derivadas simbolicamente
derivadas = [f_expr]
for i in range(6):
    derivadas.append(sp.diff(derivadas[-1], t))

# Substitui todas as derivadas de x por f_expr recursivamente
def substitute_all(expr, f_expr, max_order=6):
    for k in range(max_order, 0, -1):
        d = sp.Derivative(x, (t, k))
        if k == 1:
            expr = expr.subs(d, f_expr)
        else:
            # Substitui x^{(k)}(t) por derivada de ordem k-1 de f_expr
            expr = expr.subs(d, sp.diff(f_expr, t, k-1))
    return expr

# Aplica a substituição recursivamente em todas as derivadas
derivadas_sub = [substitute_all(d, f_expr, max_order=6) for d in derivadas]

# Converte as expressões simbólicas para funções numéricas
derivadas_calculadas = [sp.lambdify((t, x), d, 'numpy') for d in derivadas_sub]


In [ ]:
resultados_mst = {}
ordens_taylor = [2, 3, 4, 6]

for ordem in ordens_taylor:
    for dt in h:
        t_out, y_taylor = m.metodo_serie_taylor(derivadas_calculadas[:ordem], condicao_inicial, T, dt)
        y_exato = x_exato(t_out)
        dados = pd.DataFrame({
            't': t_out,
            'x_exato(t)': y_exato,
            'x(t)': y_taylor,
            'err_loc': m.erro_local(y_exato, y_taylor),
            'err_rel (%)': m.erro_relativo(y_exato, y_taylor) * 100
        })
        key = f'Taylor ordem={ordem} h={dt}'
        resultados_mst[key] = dados

        mascara = dados['t'].round(5).isin(tempos_desejados)
        codigo_latex = dados[mascara].to_latex(
            escape=True,
            index=False,
            caption=f"Resultados da Série de Taylor ordem {ordem} com h={dt} no intervalo [0, {T}].",
            label=f"tab:taylor_ordem_{ordem}_h_{dt}",
            column_format="ccccc",
            float_format="%.9f"
        )

        codigo_latex = (
            "\\begin{table}[H]\n"
            "\\centering\n"
            + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
            + "\\end{table}"
        )

        nome_arquivo = f"{diretorio_tabela}/tabela_taylor_ordem_{ordem}_h_{dt}.tex"
        with open(nome_arquivo, 'w') as file:
            file.write(codigo_latex)
        print(f"✅ Tabela Taylor ordem {ordem} h={dt} salva em: {nome_arquivo}")

fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()
t_curva_exata = np.linspace(0, T, 400)
y_curva_exata = x_exato(t_curva_exata)
for i, ordem in enumerate(ordens_taylor):
    ax = axs[i]
    keys = [k for k in resultados_mst.keys() if k.startswith(f'Taylor ordem={ordem}')]
    ax.plot(t_curva_exata, y_curva_exata, 'k-', linewidth=1.8, label='Solução Aproximada (RK6)')
    for k in keys:
        df = resultados_mst[k]
        ax.plot(df['t'], df['x(t)'], '.-', label=k)
    ax.set_xlabel('t')
    ax.set_ylabel('x(t)')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend(fontsize=10)
    ax.autoscale(enable=True, axis='x', tight=True)


nome_arquivo = f"{diretorio_imagem}/taylor_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico Taylor 2x2 salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método Runge-Kutta

In [ ]:
resultados_mrk = {}
metodos = {'RK2': m.metodo_rk2, 'RK3': m.metodo_rk3, 'RK4': m.metodo_rk4, 'RK6': m.metodo_rk6}
for nome_metodo, metodo in metodos.items():
    for dt in h:
        t_rk, x_rk = metodo(f, condicao_inicial, T, dt)
        y_exato = x_exato(t_rk)
        dados = pd.DataFrame({
            't': t_rk,
            'x_exato(t)': y_exato,
            'x(t)': x_rk,
            'err_loc': m.erro_local(y_exato, x_rk),
            'err_rel (%)': m.erro_relativo(y_exato, x_rk) * 100
        })
        resultados_mrk[f'{nome_metodo} h={dt}'] = dados


for nome, df in resultados_mrk.items():
    mascara = df['t'].round(5).isin(tempos_desejados)
    codigo_latex = df[mascara].to_latex(
        escape=True,
        index=False,
        caption=f"Resultados da simulação RK para {nome} no intervalo [0, {T}].",
        label=f"tab:{nome.replace('=', '_').replace(' ', '_')}",
        column_format="ccccc",
        float_format="%.9f"
    )

    codigo_latex = (
        "\\begin{table}[H]\n"
        "\\centering\n"
        + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
        + "\\end{table}"
    )

    nome_arquivo = f"{diretorio_tabela}/tabela_{nome.replace('=', '_').replace(' ', '_')}.tex"

    with open(nome_arquivo, 'w') as file:
        file.write(codigo_latex)

    print(f"✅ Tabela salva com sucesso no arquivo: {nome_arquivo}")

orders = ['RK2', 'RK3', 'RK4', 'RK6']
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()

t_curva_exata = np.linspace(0, T, 400)
y_curva_exata = x_exato(t_curva_exata)

for i, order in enumerate(orders):
    ax = axs[i]
    keys = [k for k in resultados_mrk.keys() if k.startswith(order)]
    ax.plot(t_curva_exata, y_curva_exata, 'k-', linewidth=1.8, label='Solução Aproximada (RK6)')
    for k in keys:
        df = resultados_mrk[k]
        ax.plot(df['t'], df['x(t)'], '.-', label=k)
    ax.set_xlabel('t')
    ax.set_ylabel('x(t)')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend(fontsize=10)
    ax.autoscale(enable=True, axis='x', tight=True)

nome_arquivo = f"{diretorio_imagem}/rk_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico 2x2 salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método Multi-Passos

In [ ]:
resultados_mmp = {}
T = .5
metodos = {'AB2': m.metodo_ab2, 'AB3': m.metodo_ab3, 'AB4': m.metodo_ab4, 'AB6': m.metodo_ab6}
for nome_metodo, metodo in metodos.items():
    for dt in h:
        t_ab, x_ab = metodo(f, condicao_inicial, T, dt)
        y_exato = x_exato(t_ab)
        dados = pd.DataFrame({
            't': t_ab,
            'x_exato(t)': y_exato,
            'x(t)': x_ab,
            'err_loc': m.erro_local(y_exato, x_ab),
            'err_rel (%)': m.erro_relativo(y_exato, x_ab) * 100
        })
        resultados_mmp[f'{nome_metodo} h={dt}'] = dados


for nome, df in resultados_mmp.items():
    mascara = df['t'].round(5).isin(tempos_desejados)
    codigo_latex = df[mascara].to_latex(
        escape=True,
        index=False,
        caption=f"Resultados da simulação AB para {nome} no intervalo [0, {T}].",
        label=f"tab:{nome.replace('=', '_').replace(' ', '_')}",
        column_format="ccccc",
        float_format="%.9f"
    )

    codigo_latex = (
        "\\begin{table}[H]\n"
        "\\centering\n"
        + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
        + "\\end{table}"
    )

    nome_arquivo = f"{diretorio_tabela}/tabela_{nome.replace('=', '_').replace(' ', '_')}.tex"

    with open(nome_arquivo, 'w') as file:
        file.write(codigo_latex)

    print(f"✅ Tabela salva com sucesso no arquivo: {nome_arquivo}")

orders = ['AB2', 'AB3', 'AB4', 'AB6']
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.flatten()

t_curva_exata = np.linspace(0, T, 400)
y_curva_exata = x_exato(t_curva_exata)

for i, order in enumerate(orders):
    ax = axs[i]
    keys = [k for k in resultados_mmp.keys() if k.startswith(order)]
    ax.plot(t_curva_exata, y_curva_exata, 'k-', linewidth=1.8, label='Solução Aproximada (RK6)')
    for k in keys:
        df = resultados_mmp[k]
        ax.plot(df['t'], df['x(t)'], '.-', label=k)
    ax.set_xlabel('t')
    ax.set_ylabel('x(t)')
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend(fontsize=10)
    ax.autoscale(enable=True, axis='x', tight=True)

nome_arquivo = f"{diretorio_imagem}/ab_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico 2x2 salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()
